In [124]:
import pandas as pd 
import sys

In [7]:
data_dir_path = '../data/'


In [25]:
def get_nyc_zcta5():
    zcta5_file_name = 'zip_to_zcta10_nyc_revised.xls'
    zcta5_df = pd.read_excel(data_dir_path + zcta5_file_name)
    zcta5_list = [str(x) for x in set(zcta5_df['zcta5'])]
    return zcta5_list
zcta5_list = get_nyc_zcta5()
print('length of zcta5_list: ', len(zcta5_list))
print('zcta5_list[:10]: ', zcta5_list[:10])

length of zcta5_list:  214
zcta5_list[:10]:  ['10271', '10278', '10279', '10280', '10282', '10301', '10302', '10303', '10304', '10305']


In [46]:
def get_nyc_only_df(input_df, zcta5_list):
    nyc_geo_list = ['8600000US' + x for x in zcta5_list]
    print('nyc_geo_list[:10]: ', nyc_geo_list[:10])
    # print(input_df['GEO_ID'].isin(nyc_geo_list))
    nyc_only_df = input_df[input_df['GEO_ID'].isin(nyc_geo_list)]
    nyc_only_df.index = range(len(nyc_only_df)) # set the index from 0
    print()
    print('nyc_only_df rows: ', nyc_only_df.shape[0])
    return nyc_only_df

### check the population: B01001
the total population of these 214 zip code ares is 8551905

In [76]:
# read data
B01001_dir_path = 'ACSDT5Y2018.B01001_2020-04-15T120015/'
B01001_file_name = 'ACSDT5Y2018.B01001_data_with_overlays_2020-04-15T120001.csv'
B01001_df = pd.read_csv(data_dir_path + B01001_dir_path + B01001_file_name)

# get nyc only data frame
B01001_nyc_only_df = get_nyc_only_df(B01001_df, zcta5_list)

# get the needed features(columns)
temp_total_population_num_feature = 'B01001_001E' # Estimate!!Total

# initialization
population_dict = {}

# result data frame
for i, row in B01001_nyc_only_df.iterrows():
    # we need to record the population of different zip code areas,
    # Since this is the first time to encounter population in different zip code areas.
    if row['GEO_ID'] in population_dict:
        print('we already have one')
    population_dict[row['GEO_ID']] = row[temp_total_population_num_feature]
    
# test if we get the right population
# print('population_dict: ', population_dict)
print('sum of population dict value: ', sum([v for _, v in population_dict.items()]))


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
sum of population dict value:  8551905


/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1. Ratio of women to men (all ages): B05003

In [110]:
# read data
B05003_dir_path = 'ACSDT5Y2018.B05003_2020-04-15T173748/'
B05003_file_name = 'ACSDT5Y2018.B05003_data_with_overlays_2020-04-15T173740.csv'
B05003_df = pd.read_csv(data_dir_path + B05003_dir_path + B05003_file_name)

# get nyc only data frame
B05003_nyc_only_df = get_nyc_only_df(B05003_df, zcta5_list)

# get the needed features(columns)
male_num_feature = 'B05003_002E' # Estimate!!Total!!Male
female_num_feature = 'B05003_013E' # Estimate!!Total!!Female

# result data frame
result_df = pd.DataFrame(columns=['GEO_ID', 'NAME', 'Ratio_women_to_men'])
count_exception = 0
for i, row in B05003_nyc_only_df.iterrows():
    try:
        result_df.loc[i] = [row['GEO_ID'], row['NAME'], row[female_num_feature]/row[male_num_feature]]
    except:
        # print(i)
        # print('row[female_num_feature]: ', row[female_num_feature])
        # print('row[male_num_feature]: ', row[male_num_feature])
        count_exception += 1
        result_df.loc[i] = [row['GEO_ID'], row['NAME'], 'None'] # Both row[female_num_feature] and row[male_num_feature] is 0

print('count_exception: ', count_exception)
result_df.head()
        

/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men
0,8600000US11222,ZCTA5 11222,1.02733
1,8600000US11233,ZCTA5 11233,1.18882
2,8600000US11235,ZCTA5 11235,1.0872
3,8600000US11426,ZCTA5 11426,1.00241
4,8600000US11427,ZCTA5 11427,1.0223


### 2. Percent Hispanic or Latino of any race: B03002

In [111]:
# read data
B03002_dir_path = 'ACSDT5Y2018.B03002_2020-04-15T173950/'
B03002_file_name = 'ACSDT5Y2018.B03002_data_with_overlays_2020-04-15T173937.csv'
B03002_df = pd.read_csv(data_dir_path + B03002_dir_path + B03002_file_name)

# get nyc only data frame
B03002_nyc_only_df = get_nyc_only_df(B03002_df, zcta5_list)

# get the needed features(columns)
population_num_feature = 'B03002_001E' # Estimate!!Total
hispanic_or_latino_num_feature = 'B03002_012E' # Estimate!!Total!!Hispanic or Latino

# double check the population: initialization
population_dict_2 = {}

# result data frame
result_df['Percent_hispanic_or_latino'] = 'None'
count_exception = 0
for i, row in B03002_nyc_only_df.iterrows():
    try:
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_hispanic_or_latino'] = row[hispanic_or_latino_num_feature]/row[population_num_feature]
    except:
        # print(i)
        # print('row[hispanic_or_latino_num_feature]: ', row[hispanic_or_latino_num_feature])
        # print('row[total_ethnicity_num_feature]: ', row[total_ethnicity_num_feature])
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_hispanic_or_latino'] = 'None' # Both value are zeros
    
    # double check the population
    if row['GEO_ID'] in population_dict_2:
        print('we already have one')
    population_dict_2[row['GEO_ID']] = row[population_num_feature]

print('count_exception: ', count_exception)

# test if we get the right population
# print('population_dict_2: ', population_dict_2)
print('sum of population dict value: ', sum([v for _, v in population_dict_2.items()])) # 8551905 is as same as before
    
result_df.head()


/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,4,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29
sum of population dict value:  8551905


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino
0,8600000US11222,ZCTA5 11222,1.02733,0.142415
1,8600000US11233,ZCTA5 11233,1.18882,0.159804
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816
3,8600000US11426,ZCTA5 11426,1.00241,0.189318
4,8600000US11427,ZCTA5 11427,1.0223,0.194533


### 3. Percent white (alone or in combination): B02008

In [112]:
# read data
B02008_dir_path = 'ACSDT5Y2018.B02008_2020-04-15T181228/'
B02008_file_name = 'ACSDT5Y2018.B02008_data_with_overlays_2020-04-15T181224.csv'
B02008_df = pd.read_csv(data_dir_path + B02008_dir_path + B02008_file_name)

# get nyc only data frame
B02008_nyc_only_df = get_nyc_only_df(B02008_df, zcta5_list)
B02008_nyc_only_df

# get the needed features(columns)
white_num_feature = 'B02008_001E' # Estimate!!Total

# print('population_dict: ', population_dict)
# # result data frame
result_df['Percent_white'] = 'None'
count_exception = 0
for i, row in B02008_nyc_only_df.iterrows():   
    try:
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_white'] = float(row[white_num_feature])/population_dict[row['GEO_ID']]
    except:
        # print(i)
        # print('row[white_num_feature]: ', row[white_num_feature])
        # print('population_dict[row[GEO_ID]]: ', population_dict[row['GEO_ID']])
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_white'] = 'None' # Both value are zeros

print('count_exception: ', count_exception)
result_df.head()


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646


### 4. Percent black or African American (alone or in combination): B02009

In [113]:
# read data
B02009_dir_path = 'ACSDT5Y2018.B02009_2020-04-15T180039/'
B02009_file_name = 'ACSDT5Y2018.B02009_data_with_overlays_2020-04-15T180030.csv'
B02009_df = pd.read_csv(data_dir_path + B02009_dir_path + B02009_file_name)

# get nyc only data frame
B02009_nyc_only_df = get_nyc_only_df(B02009_df, zcta5_list)
B02009_nyc_only_df

# get the needed features(columns)
black_num_feature = 'B02009_001E' # Estimate!!Total

# result data frame
result_df['Percent_black'] = 'None'
count_exception = 0
for i, row in B02009_nyc_only_df.iterrows():   
    try:
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_black'] = float(row[black_num_feature])/population_dict[row['GEO_ID']]
    except:
        # print(i)
        # print('row[black_num_feature]: ', row[black_num_feature])
        # print('population_dict[row[GEO_ID]]: ', population_dict[row['GEO_ID']])
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_black'] = 'None' # Both value are zeros

print('count_exception: ', count_exception)
result_df.head()


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white,Percent_black
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516,0.0543407
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069,0.803825
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574,0.0294773
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394,0.0756214
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646,0.220909


### 5. Percent asian (alone or in combination): B02011

In [115]:
# read data
B02011_dir_path = 'ACSDT5Y2018.B02011_2020-04-15T182824/'
B02011_file_name = 'ACSDT5Y2018.B02011_data_with_overlays_2020-04-15T182821.csv'
B02011_df = pd.read_csv(data_dir_path + B02011_dir_path + B02011_file_name)

# get nyc only data frame
B02011_nyc_only_df = get_nyc_only_df(B02011_df, zcta5_list)
B02011_nyc_only_df

# get the needed features(columns)
asian_num_feature = 'B02011_001E' # Estimate!!Total

# result data frame
result_df['Percent_asian'] = 'None'
count_exception = 0
for i, row in B02011_nyc_only_df.iterrows():   
    try:
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_asian'] = float(row[asian_num_feature])/population_dict[row['GEO_ID']]
    except:
        # print(i)
        # print('row[asian_num_feature]: ', row[asian_num_feature])
        # print('population_dict[row[GEO_ID]]: ', population_dict[row['GEO_ID']])
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Percent_asian'] = 'None' # Both value are zeros

print('count_exception: ', count_exception)
result_df.head()


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white,Percent_black,Percent_asian
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516,0.0543407,0.0712211
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069,0.803825,0.0231063
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574,0.0294773,0.165728
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394,0.0756214,0.464112
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646,0.220909,0.35799


### 6. Median household income: B19013

In [117]:
# read data
B19013_dir_path = 'ACSDT5Y2018.B19013_2020-04-15T183506/'
B19013_file_name = 'ACSDT5Y2018.B19013_data_with_overlays_2020-04-15T183459.csv'
B19013_df = pd.read_csv(data_dir_path + B19013_dir_path + B19013_file_name)

# get nyc only data frame
B19013_nyc_only_df = get_nyc_only_df(B19013_df, zcta5_list)
B19013_nyc_only_df

# get the needed features(columns)
median_household_income_num_feature = 'B19013_001E' # Estimate!!Total

# result data frame
result_df['median_household_income'] = 'None'
count_exception = 0
for i, row in B19013_nyc_only_df.iterrows():   
    result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'median_household_income'] = row[median_household_income_num_feature]

print('count_exception: ', count_exception)
result_df.head()


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  0


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white,Percent_black,Percent_asian,median_household_income
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516,0.0543407,0.0712211,85111
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069,0.803825,0.0231063,40449
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574,0.0294773,0.165728,52538
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394,0.0756214,0.464112,88803
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646,0.220909,0.35799,74581


### 7. Ratio of Females 16 yrs and older in labor force out of all 16 yrs and older in labor force: B23001

In [123]:
def get_labor_male_and_female_features():
    # labor male
    labor_male_feature_1 = 'B23001_004E' # Estimate!!Total!!Male!!16 to 19 years!!In labor force
    labor_male_feature_2 = 'B23001_011E' # Estimate!!Total!!Male!!20 and 21 years!!In labor force
    labor_male_feature_3 = 'B23001_018E' # Estimate!!Total!!Male!!22 to 24 years!!In labor force
    labor_male_feature_4 = 'B23001_025E' # Estimate!!Total!!Male!!25 to 29 years!!In labor force
    labor_male_feature_5 = 'B23001_032E' # Estimate!!Total!!Male!!30 to 34 years!!In labor force
    labor_male_feature_6 = 'B23001_039E' # Estimate!!Total!!Male!!35 to 44 years!!In labor force
    labor_male_feature_7 = 'B23001_046E' # Estimate!!Total!!Male!!45 to 54 years!!In labor force
    labor_male_feature_8 = 'B23001_053E' # Estimate!!Total!!Male!!55 to 59 years!!In labor force
    labor_male_feature_9 = 'B23001_060E' # Estimate!!Total!!Male!!60 and 61 years!!In labor force
    labor_male_feature_10 = 'B23001_067E' # Estimate!!Total!!Male!!62 to 64 years!!In labor force
    labor_male_feature_11 = 'B23001_074E' # Estimate!!Total!!Male!!65 to 69 years!!In labor force
    labor_male_feature_12 = 'B23001_079E' # Estimate!!Total!!Male!!70 to 74 years!!In labor force
    labor_male_feature_13 = 'B23001_084E' # Estimate!!Total!!Male!!75 years and over!!In labor force
    
    labor_male_features = []
    labor_male_features.append(labor_male_feature_1)
    labor_male_features.append(labor_male_feature_2)
    labor_male_features.append(labor_male_feature_3)
    labor_male_features.append(labor_male_feature_4)
    labor_male_features.append(labor_male_feature_5)
    labor_male_features.append(labor_male_feature_6)
    labor_male_features.append(labor_male_feature_7)
    labor_male_features.append(labor_male_feature_8)
    labor_male_features.append(labor_male_feature_9)
    labor_male_features.append(labor_male_feature_10)
    labor_male_features.append(labor_male_feature_11)
    labor_male_features.append(labor_male_feature_12)
    labor_male_features.append(labor_male_feature_13)

    # labor female
    labor_female_feature_1 = 'B23001_090E' # Estimate!!Total!!Female!!16 to 19 years!!In labor force
    labor_female_feature_2 = 'B23001_097E' # Estimate!!Total!!Female!!20 and 21 years!!In labor force
    labor_female_feature_3 = 'B23001_104E' # Estimate!!Total!!Female!!22 to 24 years!!In labor force
    labor_female_feature_4 = 'B23001_111E' # Estimate!!Total!!Female!!25 to 29 years!!In labor force
    labor_female_feature_5 = 'B23001_118E' # Estimate!!Total!!Female!!30 to 34 years!!In labor force
    labor_female_feature_6 = 'B23001_125E' # Estimate!!Total!!Female!!35 to 44 years!!In labor force
    labor_female_feature_7 = 'B23001_132E' # Estimate!!Total!!Female!!45 to 54 years!!In labor force
    labor_female_feature_8 = 'B23001_139E' # Estimate!!Total!!Female!!55 to 59 years!!In labor force
    labor_female_feature_9 = 'B23001_146E' # Estimate!!Total!!Female!!60 and 61 years!!In labor force
    labor_female_feature_10 = 'B23001_153E' # Estimate!!Total!!Female!!62 to 64 years!!In labor force
    labor_female_feature_11 = 'B23001_160E' # Estimate!!Total!!Female!!65 to 69 years!!In labor force
    labor_female_feature_12 = 'B23001_165E' # Estimate!!Total!!Female!!70 to 74 years!!In labor force
    labor_female_feature_13 = 'B23001_170E' # Estimate!!Total!!Female!!75 years and over!!In labor force
    
    labor_female_features = []
    labor_female_features.append(labor_female_feature_1)
    labor_female_features.append(labor_female_feature_2)
    labor_female_features.append(labor_female_feature_3)
    labor_female_features.append(labor_female_feature_4)
    labor_female_features.append(labor_female_feature_5)
    labor_female_features.append(labor_female_feature_6)
    labor_female_features.append(labor_female_feature_7)
    labor_female_features.append(labor_female_feature_8)
    labor_female_features.append(labor_female_feature_9)
    labor_female_features.append(labor_female_feature_10)
    labor_female_features.append(labor_female_feature_11)
    labor_female_features.append(labor_female_feature_12)
    labor_female_features.append(labor_female_feature_13)
    
    return labor_male_features, labor_female_features

# not labor
# not_labor_male_feature_1 = 'B23001_009E' # Estimate!!Total!!Male!!16 to 19 years!!Not in labor force
# not_labor_male_feature_2 = 'B23001_016E' # Estimate!!Total!!Male!!20 and 21 years!!Not in labor force
# not_labor_male_feature_3 = 'B23001_023E' # Estimate!!Total!!Male!!22 to 24 years!!Not in labor force
# not_labor_male_feature_4 = 'B23001_030E' # Estimate!!Total!!Male!!25 to 29 years!!Not in labor force
# not_labor_male_feature_5 = 'B23001_037E' # Estimate!!Total!!Male!!30 to 34 years!!Not in labor force
# not_labor_male_feature_6 = 'B23001_044E' # Estimate!!Total!!Male!!35 to 44 years!!Not in labor force
# not_labor_male_feature_7 = 'B23001_051E' # Estimate!!Total!!Male!!45 to 54 years!!Not in labor force
# not_labor_male_feature_8 = 'B23001_058E' # Estimate!!Total!!Male!!55 to 59 years!!Not in labor force


In [128]:
def get_ratio_female_labor_per_row(row, labor_male_features, labor_female_features):
    # the length for male labor and female labor should be equal
    if len(labor_male_features) != len(labor_female_features):
        sys.exit(-1)
    
    # get the sum number for male and female
    male_labor_16_yrs_and_older = 0
    female_labor_16_yrs_and_older = 0
    for i in range(len(labor_male_features)):
        male_labor_16_yrs_and_older += row[labor_male_features[i]]
        female_labor_16_yrs_and_older += row[labor_female_features[i]]
    
    return female_labor_16_yrs_and_older/(female_labor_16_yrs_and_older + male_labor_16_yrs_and_older)

In [131]:
# read data
B23001_dir_path = 'ACSDT5Y2018.B23001_2020-04-15T183827/'
B23001_file_name = 'ACSDT5Y2018.B23001_data_with_overlays_2020-04-15T183728.csv'
B23001_df = pd.read_csv(data_dir_path + B23001_dir_path + B23001_file_name)

# get nyc only data frame
B23001_nyc_only_df = get_nyc_only_df(B23001_df, zcta5_list)
B23001_nyc_only_df

# get the needed features(columns)
labor_male_features, labor_female_features = get_labor_male_and_female_features()


# result data frame
result_df['Ratio_female_labor_16_yrs_and_older'] = 'None'
count_exception = 0
for i, row in B23001_nyc_only_df.iterrows():   
    try:
        ratio_per_row = get_ratio_female_labor_per_row(row, labor_male_features, labor_female_features)
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Ratio_female_labor_16_yrs_and_older'] = ratio_per_row
    except ZeroDivisionError:
        # print(i)
        count_exception += 1
        result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Ratio_female_labor_16_yrs_and_older'] = 'None' # Both value are zeros

print('count_exception: ', count_exception)
result_df.head()


/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,2

nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  29


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white,Percent_black,Percent_asian,median_household_income,Ratio_female_labor_16_yrs_and_older
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516,0.0543407,0.0712211,85111,0.475789
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069,0.803825,0.0231063,40449,0.552321
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574,0.0294773,0.165728,52538,0.475829
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394,0.0756214,0.464112,88803,0.44793
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646,0.220909,0.35799,74581,0.460206


### 8. Educational attainment - Bachelor degree or higher: B15003

In [133]:
# read data
B15003_dir_path = 'ACSDT5Y2018.B15003_2020-04-15T184431/'
B15003_file_name = 'ACSDT5Y2018.B15003_data_with_overlays_2020-04-15T184411.csv'
B15003_df = pd.read_csv(data_dir_path + B15003_dir_path + B15003_file_name)

# get nyc only data frame
B15003_nyc_only_df = get_nyc_only_df(B15003_df, zcta5_list)
B15003_nyc_only_df

# get the needed features(columns)
education_num_feature_1 = 'B15003_022E' # Estimate!!Total!!Bachelor's degree
education_num_feature_2 = 'B15003_023E' # Estimate!!Total!!Master's degree
education_num_feature_3 = 'B15003_024E' # Estimate!!Total!!Professional school degree
education_num_feature_4 = 'B15003_025E' # Estimate!!Total!!Doctorate degree

def get_bachelor_degree_or_higher_per_row(row):
    return row[education_num_feature_1] + row[education_num_feature_2] + row[education_num_feature_3] + row[education_num_feature_4]

# result data frame
result_df['Bachelor_degree_or_higher'] = 'None'
count_exception = 0
for i, row in B15003_nyc_only_df.iterrows():   
    value_per_row = get_bachelor_degree_or_higher_per_row(row)
    result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Bachelor_degree_or_higher'] = value_per_row

print('count_exception: ', count_exception)
result_df.head()


/Users/meettyj/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  0


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white,Percent_black,Percent_asian,median_household_income,Ratio_female_labor_16_yrs_and_older,Bachelor_degree_or_higher
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516,0.0543407,0.0712211,85111,0.475789,17728
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069,0.803825,0.0231063,40449,0.552321,12510
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574,0.0294773,0.165728,52538,0.475829,28403
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394,0.0756214,0.464112,88803,0.44793,5501
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646,0.220909,0.35799,74581,0.460206,5571


### 9. Average household size: B25010

In [135]:
# read data
B25010_dir_path = 'ACSDT5Y2018.B25010_2020-04-17T172107/'
B25010_file_name = 'ACSDT5Y2018.B25010_data_with_overlays_2020-04-17T172102.csv'
B25010_df = pd.read_csv(data_dir_path + B25010_dir_path + B25010_file_name)

# get nyc only data frame
B25010_nyc_only_df = get_nyc_only_df(B25010_df, zcta5_list)
B25010_nyc_only_df

# get the needed features(columns)
average_household_size_feature = 'B25010_001E' # Estimate!!Total

# result data frame
result_df['Average_household_size'] = 'None'
count_exception = 0
for i, row in B25010_nyc_only_df.iterrows():   
    result_df.loc[result_df['GEO_ID'] == row['GEO_ID'], 'Average_household_size'] = row[average_household_size_feature]

print('count_exception: ', count_exception)
result_df.head()


nyc_geo_list[:10]:  ['8600000US10271', '8600000US10278', '8600000US10279', '8600000US10280', '8600000US10282', '8600000US10301', '8600000US10302', '8600000US10303', '8600000US10304', '8600000US10305']

nyc_only_df rows:  214
count_exception:  0


,GEO_ID,NAME,Ratio_women_to_men,Percent_hispanic_or_latino,Percent_white,Percent_black,Percent_asian,median_household_income,Ratio_female_labor_16_yrs_and_older,Bachelor_degree_or_higher,Average_household_size
0,8600000US11222,ZCTA5 11222,1.02733,0.142415,0.856516,0.0543407,0.0712211,85111,0.475789,17728,2.10
1,8600000US11233,ZCTA5 11233,1.18882,0.159804,0.101069,0.803825,0.0231063,40449,0.552321,12510,2.57
2,8600000US11235,ZCTA5 11235,1.0872,0.0830816,0.77574,0.0294773,0.165728,52538,0.475829,28403,2.38
3,8600000US11426,ZCTA5 11426,1.00241,0.189318,0.403394,0.0756214,0.464112,88803,0.44793,5501,3.26
4,8600000US11427,ZCTA5 11427,1.0223,0.194533,0.310646,0.220909,0.35799,74581,0.460206,5571,3.08


--------------------------
## Finally: save to CSV